In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Wczytanie danych z pliku
file_path = 'pol.txt'
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Ograniczenie do pierwszych 10 000 zdań
max_samples = 10000
lines = lines[:max_samples]

# Podzielenie na listy zdań w języku angielskim i polskim
input_texts = []  # Zdania w języku angielskim
target_texts = []  # Zdania w języku polskim

for line in lines:
    try:
        en, pl, _ = line.split('\t') 
        input_texts.append(en.strip().split())
        target_texts.append(['<start>'] + pl.strip().split() + ['<end>'])
    except ValueError:
        continue 

# Sprawdzenie wczytanych danych
print(f"Przykładowe zdanie w języku angielskim: {input_texts[0]}")
print(f"Przykładowe zdanie w języku polskim: {target_texts[0]}")
print(f"Łączna liczba wczytanych par zdań: {len(input_texts)}")


Przykładowe zdanie w języku angielskim: ['Go.']
Przykładowe zdanie w języku polskim: ['<start>', 'Idź.', '<end>']
Łączna liczba wczytanych par zdań: 10000


In [2]:
# Utworzenie słowników
input_vocab = set(word for sentence in input_texts for word in sentence)
target_vocab = set(word for sentence in target_texts for word in sentence)

input_word2idx = {word: idx for idx, word in enumerate(sorted(input_vocab))}
target_word2idx = {word: idx for idx, word in enumerate(sorted(target_vocab))}

input_idx2word = {idx: word for word, idx in input_word2idx.items()}
target_idx2word = {idx: word for word, idx in target_word2idx.items()}

# Sprawdzenie długości słowników
print(f"Liczba unikalnych słów w języku angielskim: {len(input_vocab)}")
print(f"Liczba unikalnych słów w języku polskim: {len(target_vocab)}")

# Maksymalna długość sekwencji
max_input_seq_length = max(len(sentence) for sentence in input_texts)
max_target_seq_length = max(len(sentence) for sentence in target_texts)

print(f"Maksymalna długość sekwencji w języku angielskim: {max_input_seq_length}")
print(f"Maksymalna długość sekwencji w języku polskim: {max_target_seq_length}")


Liczba unikalnych słów w języku angielskim: 4485
Liczba unikalnych słów w języku polskim: 8001
Maksymalna długość sekwencji w języku angielskim: 6
Maksymalna długość sekwencji w języku polskim: 11


In [3]:
# Przygotowanie danych wejściowych i docelowych
encoder_input_data = [
    [input_word2idx[word] for word in sentence] for sentence in input_texts
]
decoder_input_data = [
    [target_word2idx[word] for word in sentence] for sentence in target_texts
]
decoder_target_data = [
    sentence[1:] for sentence in decoder_input_data 
]

# Uzupełnianie sekwencji paddingiem
encoder_input_data = pad_sequences(encoder_input_data, maxlen=max_input_seq_length, padding="post")
decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_target_seq_length, padding="post")
decoder_target_data = pad_sequences(decoder_target_data, maxlen=max_target_seq_length, padding="post")

# One-hot encoding danych docelowych
decoder_target_data = to_categorical(decoder_target_data, num_classes=len(target_vocab))

print("Dane wejściowe i docelowe zostały przygotowane.")


Dane wejściowe i docelowe zostały przygotowane.


In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

# Liczba jednostek w warstwach LSTM
latent_dim = 256

# Enkoder
encoder_inputs = Input(shape=(max_input_seq_length,))  
embedded_encoder_inputs = tf.keras.layers.Embedding(input_dim=len(input_vocab), output_dim=latent_dim)(encoder_inputs)

# Warstwy LSTM z Dropout
encoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.2)
encoder_outputs, _, _ = encoder_lstm1(embedded_encoder_inputs)

encoder_lstm2 = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.2)
encoder_outputs, _, _ = encoder_lstm2(encoder_outputs)

encoder_lstm3 = LSTM(latent_dim, return_sequences=False, return_state=True, dropout=0.2)
encoder_outputs, state_h, state_c = encoder_lstm3(encoder_outputs)

# Stany enkodera (do przekazania do dekodera)
encoder_states = [state_h, state_c]

# Callback EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Model enkodera
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

# Wyświetlenie struktury enkodera
encoder_model.summary()



Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, 6, 256)         │     1,148,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ [(None, 6, 256),       │       525,312 │
│                                 │ (None, 256), (None,    │               │
│                                 │ 256)]                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ [(None, 6, 256),       │       525,312 │
│                                 │ (None, 256), (None,    │               │
│                                 │ 256)]                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ [(None, 256), (None,   │       525,312 │
│                                 │ 256), (None, 256)]     │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,724,096 (10.39 MB)

 Trainable params: 2,724,096 (10.39 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from tensorflow.keras.layers import Embedding, Dense

# Dekoder
decoder_inputs = Input(shape=(max_target_seq_length,))
embedded_decoder_inputs = Embedding(input_dim=len(target_vocab), output_dim=latent_dim)(decoder_inputs)

# Warstwy LSTM z Dropout
decoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.2)
decoder_outputs, _, _ = decoder_lstm1(embedded_decoder_inputs, initial_state=encoder_states)

decoder_lstm2 = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.2)
decoder_outputs, _, _ = decoder_lstm2(decoder_outputs)

decoder_lstm3 = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.2)
decoder_outputs, _, _ = decoder_lstm3(decoder_outputs)

# Warstwa Dense dla przewidywania słów
decoder_dense = Dense(len(target_vocab), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Tworzenie pełnego modelu Seq2Seq
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Wyświetlenie struktury modelu Seq2Seq
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 6, 256)    │  1,148,160 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 6, 256),  │    525,312 │ embedding[0][0]   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 6, 256),  │    525,312 │ lstm[0][0]        │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 11, 256)   │  2,048,256 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 256),     │    525,312 │ lstm_1[0][0]      │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, 11, 256), │    525,312 │ embedding_1[0][0… │
│                     │ (None, 256),      │            │ lstm_2[0][1],     │
│                     │ (None, 256)]      │            │ lstm_2[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 11, 256), │    525,312 │ lstm_3[0][0]      │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 11, 256), │    525,312 │ lstm_4[0][0]      │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 11, 8001)  │  2,056,257 │ lstm_5[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,404,545 (32.06 MB)

 Trainable params: 8,404,545 (32.06 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()



Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 6, 256)    │  1,148,160 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 6, 256),  │    525,312 │ embedding[0][0]   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 6, 256),  │    525,312 │ lstm[0][0]        │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 11, 256)   │  2,048,256 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 256),     │    525,312 │ lstm_1[0][0]      │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, 11, 256), │    525,312 │ embedding_1[0][0… │
│                     │ (None, 256),      │            │ lstm_2[0][1],     │
│                     │ (None, 256)]      │            │ lstm_2[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 11, 256), │    525,312 │ lstm_3[0][0]      │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 11, 256), │    525,312 │ lstm_4[0][0]      │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 11, 8001)  │  2,056,257 │ lstm_5[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,404,545 (32.06 MB)

 Trainable params: 8,404,545 (32.06 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Trening modelu Seq2Seq z informacjami o postępie
history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=50,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1 
)

print(f"Najniższa strata walidacyjna: {min(history.history['val_loss']):.4f}")
if 'val_accuracy' in history.history:
    print(f"Najwyższa dokładność walidacyjna: {max(history.history['val_accuracy']):.4f}")

import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss During Training')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

if 'accuracy' in history.history:
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy During Training')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()


Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 94s 565ms/step - accuracy: 0.6501 - loss: 4.2936 - val_accuracy: 0.6450 - val_loss: 2.6467
Epoch 2/50
 34/125 ━━━━━━━━━━━━━━━━━━━━ 44s 487ms/step - accuracy: 0.7266 - loss: 1.9575

KeyboardInterrupt: 

In [ ]:
def decode_sequence(input_seq):
    # Kodowanie danych wejściowych
    states_value = encoder_model.predict(input_seq)
    
    # Zainicjowanie sekwencji dekodera (z tokenem <start>)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_word2idx['<start>']
    
    # Dekodowanie iteracyjne
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # Wybranie przewidywanego słowa
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = target_idx2word[sampled_token_index]
        decoded_sentence.append(sampled_word)
        
        # Wyjście z pętli, gdy osiągnięty zostanie <end> lub maksymalna długość
        if sampled_word == '<end>' or len(decoded_sentence) > max_target_seq_length:
            stop_condition = True
        
        # Zaktualizowanie sekwencji wejściowej dekodera
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        # Zaktualizowanie stanów ukrytych
        states_value = [h, c]
    
    return ' '.join(decoded_sentence[:-1])  # Pomijamy token <end> w wyniku

# Przykładowe testy
for seq_index in range(5):  # Testujemy na pierwszych 5 zdaniach
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    print("Input sentence:", ' '.join(input_texts[seq_index]))
    print("Decoded sentence:", decoded_sentence)
    print("Target sentence:", ' '.join(target_texts[seq_index][1:-1]))  # Pomijamy <start> i <end>
    print("-" * 50)
